In [647]:
# Day 1
import numpy as np

with open("day1input") as file:
    modules = [int(x) for x in file.read().split(",")]


def fuel_by_mass(mass):
    return (mass // 3) - 2


@np.vectorize
def calculate_module_fuel(module_weight):
    total = fuel_by_mass(module_weight)
    fuel_mass = fuel_by_mass(total)

    while fuel_mass > 0:
        total += fuel_mass
        fuel_mass = fuel_by_mass(fuel_mass)

    return total


required_fuel = 0

for module_weight in modules:
    required_fuel += calculate_fuel(module_weight)

sum(calculate_module_fuel(modules))

4998585

In [646]:
# Day 2
import operator


def run_intcode(intcode):
    opcodes = {1: operator.add, 2: operator.mul}

    for i in range(0, len(intcode), 4):
        if intcode[i] == 99:
            return intcode
        elif intcode[i] in opcodes:
            opcode, L1, L2, out = intcode[i : i + 4]
            intcode[out] = opcodes.get(opcode)(intcode[L1], intcode[L2])
        else:
            raise ValueError("Error")


with open("day2input") as file:
    ip = [int(x) for x in file.read().split(",")]

run_intcode(ip)

[19690720,
 45,
 59,
 2,
 1,
 1,
 2,
 3,
 1,
 3,
 4,
 3,
 1,
 5,
 0,
 3,
 2,
 1,
 10,
 180,
 1,
 6,
 19,
 182,
 2,
 23,
 6,
 364,
 2,
 6,
 27,
 728,
 2,
 13,
 31,
 3640,
 1,
 10,
 35,
 3644,
 2,
 39,
 13,
 18220,
 1,
 43,
 13,
 18225,
 1,
 6,
 47,
 18227,
 1,
 10,
 51,
 18231,
 2,
 55,
 6,
 36462,
 1,
 5,
 59,
 36463,
 2,
 9,
 63,
 109389,
 1,
 6,
 67,
 109391,
 2,
 9,
 71,
 328173,
 1,
 6,
 75,
 328175,
 2,
 79,
 13,
 1640875,
 1,
 83,
 10,
 1640879,
 1,
 13,
 87,
 1640884,
 1,
 91,
 10,
 1640888,
 2,
 9,
 95,
 4922664,
 1,
 5,
 99,
 4922665,
 2,
 10,
 103,
 19690660,
 1,
 107,
 2,
 19690719,
 1,
 111,
 5,
 0,
 99,
 2,
 14,
 0,
 0]

In [642]:
# Day 3


def build_wire(instructions):
    def parse_instruction(instruction):
        direction, *steps = instruction
        return direction, int("".join(steps))

    point = [0, 0]
    wire = set()
    steps = 0
    min_steps = {}

    for instruction in instructions:
        direction, units = parse_instruction(instruction)

        if direction in {"R", "U"}:
            sign = 1
        else:
            sign = -1

        if direction in {"L", "R"}:
            axis = 0
        else:
            axis = 1

        for _ in range(units):
            point[axis] += sign
            new_point = tuple(point)
            wire.add(new_point)

            steps += 1

            if new_point not in min_steps:
                min_steps[new_point] = steps

    return {"coordinates": wire, "steps": min_steps}


with open("day3ip1") as file:
    ip1 = file.read().split(",")

with open("day3ip2") as file:
    ip2 = file.read().split(",")

inputs = (ip1, ip2)

wires = [build_wire(ip) for ip in inputs]

crosses = wires[0]["coordinates"].intersection(wires[1]["coordinates"])

cross_steps = (wires[0]["steps"][x] + wires[1]["steps"][x] for x in crosses)

print(min(cross_steps))

15612


In [299]:
# Day 4
"""
Part 1

--- Day 4: Secure Container ---
You arrive at the Venus fuel depot only to discover it's protected by a password. The Elves had written the password on a sticky note, but someone threw it out.

However, they do remember a few key facts about the password:

It is a six-digit number.
The value is within the range given in your puzzle input.
Two adjacent digits are the same (like 22 in 122345).
Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).
Other than the range rule, the following are true:

111111 meets these criteria (double 11, never decreases).
223450 does not meet these criteria (decreasing pair of digits 50).
123789 does not meet these criteria (no double).
How many different passwords within the range given in your puzzle input meet these criteria?

Your puzzle input is 171309-643603.

"""


def possibilities(low, high):
    def has_doubled(n):
        n = str(n)
        for i, x in enumerate(n):
            if i == len(n) - 1:
                break
            elif x == n[i + 1]:
                return True

    def is_asc(n):
        n = str(n)
        for i, x in enumerate(n):
            if x > n[i + 1]:
                return False
            if i == len(n) - 2:
                break

        return True

    return [x for x in range(low, high) if has_doubled(x) and is_asc(x)]


"""
--- Part Two ---
An Elf just remembered one more important detail: the two adjacent matching digits are not part of a larger group of matching digits.

Given this additional criterion, but still ignoring the range rule, the following are now true:

112233 meets these criteria because the digits never decrease and all repeated digits are exactly two digits long.
123444 no longer meets the criteria (the repeated 44 is part of a larger group of 444).
111122 meets the criteria (even though 1 is repeated more than twice, it still contains a double 22).
How many different passwords within the range given in your puzzle input meet all of the criteria?

Your puzzle input is still 171309-643603."""


def possibilities_p2(low, high):
    def has_doubled_strict(n):
        """Looks for an occurrence of exactly two identical digits side by side"""
        n = str(n)
        for i, x in enumerate(n):
            if i == len(n) - 1:
                # If we reach the last digit without finding a double it can't be true
                break
            elif x == n[i + 1] and ((i == len(n) - 2) or x != n[i + 2]):
                # Does x match the next digit, and if so,
                # is it second to last or different from two spots ahead?
                if x == n[i - 1]:
                    # If we get here and it matches behind,
                    # it's a run of at least 3 and doesn't count
                    continue
                else:
                    return True
        return False

    def is_asc(n):
        n = str(n)
        for i, x in enumerate(n):
            if x > n[i + 1]:
                return False
            if i == len(n) - 2:
                break
        return True

    return [x for x in range(low, high) if has_doubled_strict(x) and is_asc(x)]


low = 171309
high = 643603

print(
    f"""
    P1: {len(set(possibilities(low,high)))}
    P2: {len(set(possibilities_p2(low,high)))}"""
)

P1: 1625
P2: 1111


In [639]:
# Day 5
"""--- Day 5: Sunny with a Chance of Asteroids ---
You're starting to sweat as the ship makes its way toward Mercury. The Elves suggest that you get the air conditioner working by upgrading your ship computer to support the Thermal Environment Supervision Terminal.

The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). The TEST diagnostic program will run on your existing Intcode computer after a few modifications:

First, you'll need to add two new instructions:

Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.
Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Second, you'll need to add support for parameter modes:

Each parameter of an instruction is handled based on its parameter mode. Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. Until now, all parameters have been in position mode.

Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

Parameter modes are stored in the same value as the instruction's opcode. The opcode is a two-digit number based only on the ones and tens digit of the value, that is, the opcode is the rightmost two digits of the first value in an instruction. Parameter modes are single digits, one per parameter, read right-to-left from the opcode: the first parameter's mode is in the hundreds digit, the second parameter's mode is in the thousands digit, the third parameter's mode is in the ten-thousands digit, and so on. Any missing modes are 0.

For example, consider the program 1002,4,3,4,33.

The first instruction, 1002,4,3,4, is a multiply instruction - the rightmost two digits of the first value, 02, indicate opcode 2, multiplication. Then, going right to left, the parameter modes are 0 (hundreds digit), 1 (thousands digit), and 0 (ten-thousands digit, not present and therefore zero):

ABCDE
 1002

DE - two-digit opcode,      02 == opcode 2
 C - mode of 1st parameter,  0 == position mode
 B - mode of 2nd parameter,  1 == immediate mode
 A - mode of 3rd parameter,  0 == position mode,
                                  omitted due to being a leading zero
This instruction multiplies its first two parameters. The first parameter, 4 in position mode, works like it did before - its value is the value stored at address 4 (33). The second parameter, 3 in immediate mode, simply has value 3. The result of this operation, 33 * 3 = 99, is written according to the third parameter, 4 in position mode, which also works like it did before - 99 is written to address 4.

Parameters that an instruction writes to will never be in immediate mode.

Finally, some notes:

It is important to remember that the instruction pointer should increase by the number of values in the instruction after the instruction finishes. Because of the new instructions, this amount is no longer always 4.
Integers can be negative: 1101,100,-1,4,0 is a valid program (find 100 + -1, store the result in position 4).
The TEST diagnostic program will start by requesting from the user the ID of the system to test by running an input instruction - provide it 1, the ID for the ship's air conditioner unit.

It will then perform a series of diagnostic tests confirming that various parts of the Intcode computer, like parameter modes, function correctly. For each test, it will run an output instruction indicating how far the result of the test was from the expected value, where 0 means the test was successful. Non-zero outputs mean that a function is not working correctly; check the instructions that were run before the output instruction to see which one failed.

Finally, the program will output a diagnostic code and immediately halt. This final output isn't an error; an output followed immediately by a halt means the program finished. If all outputs were zero except the diagnostic code, the diagnostic program ran successfully.

After providing 1 to the only input instruction and passing all the tests, what diagnostic code does the program produce?"""

import operator


def run_intcode_v2(intcode):
    def _value_by_mode(value, is_immediate):
        return int(value if is_immediate else intcode[value])

    opcodes = {
        1: {"op": operator.add, "args": 3},
        2: {"op": operator.mul, "args": 3},
        3: {"op": None, "args": 1},
        4: {"op": None, "args": 1},
    }

    pos = 0  # position on tape

    while True:
        opcode = intcode[pos] % 100
        argmodes = list(map(int, list(str(intcode[pos] // 100))[::-1]))
        pos += 1

        if opcode not in opcodes:
            break

        op, step = opcodes[opcode].values()

        argmodes.extend([0] * (step - len(argmodes)))
        args = intcode[pos : pos + step]

        # print(f'pos={pos}, code={opcode}, modes={argmodes}, args={args}')

        if opcode in {1, 2}:
            arg1, arg2 = (_value_by_mode(*x) for x in zip(args[:2], argmodes))
            out = args[2]

            # print(f'arg1={arg1}, arg2={arg2}, out={out}, {op}({arg1},{arg2})={op(arg1,arg2)}\n')

            intcode[out] = op(arg1, arg2)

        elif opcode == 3:
            out = args[0]
            ip = input("Enter System ID: ")
            intcode[out] = ip

        elif opcode == 4:
            arg = _value_by_mode(args[0], argmodes[0])
            print(arg)

        pos += step


"""--- Part Two ---
The air conditioner comes online! Its cold air feels good for a while, but then the TEST alarms start to go off. Since the air conditioner can't vent its heat anywhere but back into the spacecraft, it's actually making the air inside the ship warmer.

Instead, you'll need to use the TEST to extend the thermal radiators. Fortunately, the diagnostic program (your puzzle input) is already equipped for this. Unfortunately, your Intcode computer is not.

Your computer is only missing a few opcodes:

Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
Like all instructions, these instructions need to support parameter modes as described above.

Normally, after an instruction is finished, the instruction pointer increases by the number of values in that instruction. However, if the instruction modifies the instruction pointer, that value is used and the instruction pointer is not automatically increased.

For example, here are several programs that take one input, compare it to the value 8, and then produce one output:

3,9,8,9,10,9,4,9,99,-1,8 - Using position mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
3,9,7,9,10,9,4,9,99,-1,8 - Using position mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
3,3,1108,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
3,3,1107,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
Here are some jump tests that take an input, then output 0 if the input was zero or 1 if the input was non-zero:

3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9 (using position mode)
3,3,1105,-1,9,1101,0,0,12,4,12,99,1 (using immediate mode)
Here's a larger example:

3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
The above example program uses an input instruction to ask for a single number. The program will then output 999 if the input value is below 8, output 1000 if the input value is equal to 8, or output 1001 if the input value is greater than 8.

This time, when the TEST diagnostic program runs its input instruction to get the ID of the system to test, provide it 5, the ID for the ship's thermal radiator controller. This diagnostic test suite only outputs one number, the diagnostic code.

What is the diagnostic code for system ID 5?"""


def run_intcode_v3(intcode):
    def _value_by_mode(value, is_immediate):
        return int(value if is_immediate else intcode[value])

    def jump_if_true(flag, new_pos, pos):
        return new_pos if flag else pos + 2

    def jump_if_false(flag, new_pos, pos):
        return new_pos if not flag else pos + 2

    def less_than(arg1, arg2):
        return 1 if arg1 < arg2 else 0

    def equals(arg1, arg2):
        return 1 if arg1 == arg2 else 0

    opcodes = {
        1: {"op": operator.add, "args": 3},
        2: {"op": operator.mul, "args": 3},
        3: {"op": None, "args": 1},
        4: {"op": None, "args": 1},
        5: {"op": jump_if_true, "args": 2},
        6: {"op": jump_if_false, "args": 2},
        7: {"op": less_than, "args": 3},
        8: {"op": equals, "args": 3},
    }

    pos = 0  # position on tape

    while True:
        opcode = intcode[pos] % 100
        argmodes = list(map(int, list(str(intcode[pos] // 100))[::-1]))
        pos += 1

        if opcode not in opcodes:
            break

        op, step = opcodes[opcode].values()

        argmodes.extend([0] * (step - len(argmodes)))
        args = intcode[pos : pos + step]

        # print(f'pos={pos}, code={opcode}, modes={argmodes}, args={args}')

        if opcode in {1, 2}:
            arg1, arg2 = (_value_by_mode(*x) for x in zip(args[:2], argmodes))
            out = args[2]

            intcode[out] = op(arg1, arg2)

            pos += step

        elif opcode == 3:
            out = args[0]
            ip = input("Enter System ID: ")
            intcode[out] = ip
            pos += step

        elif opcode == 4:
            arg = _value_by_mode(args[0], argmodes[0])
            print(arg)
            pos += step

        elif opcode in {5, 6}:
            arg1, arg2 = (_value_by_mode(*x) for x in zip(args[:2], argmodes))
            pos = op(arg1, arg2, pos)

        elif opcode in {7, 8}:
            arg1, arg2 = (_value_by_mode(*x) for x in zip(args[:2], argmodes))
            out = args[2]
            intcode[out] = op(arg1, arg2)
            pos += step


with open("day5input") as file:
    ip = [int(x) for x in file.read().split(",")]

run_intcode_v3(ip)

Enter System ID:  5


4283952


In [614]:
# Day 6

with open("day6input", newline=None) as file:
    ip = file.read().split("\n")

orbits = {y: x for x, y in [x.split(")") for x in ip]}


def count_orbits(obj, orbits):
    if obj not in orbits:
        return 0

    return 1 + count_orbits(orbits[obj], orbits)


def indirect_orbits(obj, orbits):
    if obj not in orbits:
        return []

    return [orbits[obj]] + indirect_orbits(orbits[obj], orbits)


def all_orbits(orbits):
    total = 0

    for orbit in orbits:
        total += count_orbits(orbit, orbits)

    return total


def orbital_swaps(start, end, orbits):
    left = set(indirect_orbits(start, orbits))
    right = set(indirect_orbits(end, orbits))

    return len(left ^ right)


print(f"Total Orbits: {all_orbits(orbits)}")
print(f"Orbital swaps between you and Santa: {orbital_swaps_v2('YOU','SAN',orbits)}")

Total Orbits: 186597
Orbital swaps between you and Santa: 412
